In [12]:
print("OK")

OK


In [13]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [14]:
PINECONE_API_KEY="77a54e16-4df7-4e2a-8016-acdd738906c8"
PINECONE_API_ENV="us-west4-gcp-free"

In [15]:
def load_pdf(Data):
    loader=DirectoryLoader(Data,glob="*.pdf",loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [16]:
extracted_data=load_pdf("Data/")

In [17]:
#Create text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [19]:
text_chunks=text_split(extracted_data)
print("length of chunk:",len(text_chunks))

length of chunk: 7020


In [20]:
#download embedding model

def download_hugging_face_embedding():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [21]:
embeddings=download_hugging_face_embedding()

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 280kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 22.1kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 1.62MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 23.2kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 197kB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:14<00:00, 6.25MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 10.6kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 27.9kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 691kB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 69.8kB/s]
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00, 2.63MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 8.86MB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 87.2kB/s]


In [22]:
pinecone.init(api_key=PINECONE_API_KEY,environment=PINECONE_API_ENV)

index_name="medical-chatbot"

docsearch=Pinecone.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)



In [23]:
docsearch=Pinecone.from_existing_index(index_name,embeddings)


In [24]:
prompt_template=""""
Use the following piece of information to answer the user's question.
If you dont know the answer,just say that you dont know,dont try to make up an answer.

Context:{context}
Question={question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [25]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [26]:
llm=CTransformers(model="models/llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",config={'max_new_tokens':512,'temperature':0.8})

In [27]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [28]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response :",result["result"])

Response : Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
